## Script to Pull American Community Survey Census Data

This script utilizes the following API: 
https://pypi.org/project/CensusData/

In [2]:
import pandas as pd
import censusdata

## Fill in options in the cell below 

In [11]:
# set survey to pull from American Community Survey Estimates (options are 'acs1', 'acs3','acs5')
survey = 'acs1'

# years parameter for download 
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019]

file_name = 'census_data.xlsx'


In [4]:
# search for table ID to use in query 
censusdata.search('acs1', 2016, 'label', 'educational')

[('B08126_011E',
  'MEANS OF TRANSPORTATION TO WORK BY INDUSTRY',
  'Estimate!!Total!!Educational services, and health care and social assistance'),
 ('B08126_026E',
  'MEANS OF TRANSPORTATION TO WORK BY INDUSTRY',
  'Estimate!!Total!!Car, truck, or van - drove alone!!Educational services, and health care and social assistance'),
 ('B08126_041E',
  'MEANS OF TRANSPORTATION TO WORK BY INDUSTRY',
  'Estimate!!Total!!Car, truck, or van - carpooled!!Educational services, and health care and social assistance'),
 ('B08126_056E',
  'MEANS OF TRANSPORTATION TO WORK BY INDUSTRY',
  'Estimate!!Total!!Public transportation (excluding taxicab)!!Educational services, and health care and social assistance'),
 ('B08126_071E',
  'MEANS OF TRANSPORTATION TO WORK BY INDUSTRY',
  'Estimate!!Total!!Walked!!Educational services, and health care and social assistance'),
 ('B08126_086E',
  'MEANS OF TRANSPORTATION TO WORK BY INDUSTRY',
  'Estimate!!Total!!Taxicab, motorcycle, bicycle, or other means!!Educat


## Example Tables from American Community Survey 1-year data (ACS1)
<br>
population:
<br>
('B01003_001E', 'TOTAL POPULATION', 'Estimate!!Total')
<br>
<br>
race:
<br>
('B02001_001E', 'RACE', 'Estimate!!Total:'),
<br>
('B02001_002E', 'RACE', 'Estimate!!Total:!!White alone'),

<br>
<br>
education:
<br>
'B15003_022E':'bachelors','B15003_023E':'masters','B15003_024E':'pro_school','B15003_025E':'doctorate'
<br>
<br>
median_income:
<br>
('B19001_001E',
  'HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN x year INFLATION-ADJUSTED DOLLARS)',
  'Estimate!!Total:'),
  
  
## Other sources to help you find the table you are looking for
https://censusreporter.org/
https://api.census.gov/data/2017/acs/acs5/groups/{table_id}.html
<br>
Example:
<br>
Searching "Median Income" in census reporter to get the idea to start looking into table B19001. We can see a full breakdown of the json structure of the Census API with this link to see what columns are available in this table:
https://api.census.gov/data/2017/acs/acs5/groups/B19001.html



In [5]:
my_tables = ['B01003_001E', 'B02001_001E', 'B02001_002E','B15003_022E', 'B15003_023E','B15003_024E','B15003_025E', 'B19001_001E']

In [6]:
years_str = ([str(x) for x in years])
tables = {}
data = pd.DataFrame()

for year in years:
    
    # download table data from census
    data = censusdata.download(survey, year, censusdata.censusgeo([('state', '*')]), my_tables
                              )
    
    # set state name as the index column
    data = data.reset_index()
    data['state'] = data['index'].astype(str).str.split(':').str[0]
    data = data.drop(['index'], axis=1)
    data = data.set_index('state')
    
    # create column for year
    data['year'] = year
    
    #uncomment below to rename your columns with more descriptive names
    '''
    # rename table column headers
    data = data.rename(columns={'B01003_001E':'population','B19013_001E':'median_income', 'B02001_001E':'race_total','B02001_002E':'white_only',
                                'B15003_022E':'bachelors','B15003_023E':'masters','B15003_024E':'pro_school','B15003_025E':'doctorate'})
    '''

    # store data for each yeah in dictionary with key year{year} ex: 'year2019'
    tables["year{0}".format(year)] = data.copy()

In [7]:
data = pd.concat(tables.values())

In [8]:
data

,B01003_001E,B02001_001E,B02001_002E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B19001_001E,year
state,,,,,,,,,
Mississippi,2991207,2991207,1764640,248488,104166,23099,17653,1091002,2013
Missouri,6044171,6044171,5003971,685991,293327,63486,48201,2362853,2013
Montana,1015165,1015165,904740,135518,44501,12062,7707,406288,2013
Nebraska,1868516,1868516,1650358,238160,81414,22745,14473,730579,2013
Nevada,2790136,2790136,1906864,282283,97337,27801,14679,1002571,2013
...,...,...,...,...,...,...,...,...,...
Washington,7614893,7614893,5651091,1204728,538939,122253,89712,2932477,2019
West Virginia,1792147,1792147,1668004,161686,80380,15831,11809,728175,2019
Wisconsin,5822434,5822434,4963354,829878,307952,71262,49287,2386623,2019


In [9]:
data = data.sort_values(by='state')

In [12]:
data.to_excel(file_name)  